## Pandas Lab Assignment
#### Rachel Seo ydp7xv

#### Question: Are kickers more likely to miss the field goal when distance is 50 yds+?

#### Pseudocode 
1. Create a new dataset by filtering the current dataset for only the 'distance', 'GOOD', and 'Missed'
2. Extract all field goal attempts 
3. Separate attempts into two categories - shorter than 50 yards vs. 50 yards or more
4. Calculate miss percentage for each category 

In [1]:
# import libraries
import pandas as pd
import os
os.getcwd()

'/workspaces/DS-3021/02_function_basics'

In [2]:
# Loading and preparing the data
nfl = pd.read_csv("../data/nfl2008_fga.csv")

print(nfl.info())   # .info() outputs the structure of the dataframe object. It includes the columns, the data types in each, etc.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1039 entries, 0 to 1038
Data columns (total 23 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   GameDate  1039 non-null   int64  
 1   AwayTeam  1039 non-null   object 
 2   HomeTeam  1039 non-null   object 
 3   qtr       1039 non-null   int64  
 4   min       1039 non-null   int64  
 5   sec       1039 non-null   int64  
 6   kickteam  1039 non-null   object 
 7   def       1039 non-null   object 
 8   down      1037 non-null   float64
 9   togo      1037 non-null   float64
 10  kicker    1039 non-null   int64  
 11  ydline    1039 non-null   int64  
 12  name      1039 non-null   object 
 13  distance  1039 non-null   int64  
 14  homekick  1039 non-null   int64  
 15  kickdiff  1039 non-null   int64  
 16  timerem   1039 non-null   int64  
 17  offscore  1039 non-null   int64  
 18  defscore  1039 non-null   int64  
 19  season    1039 non-null   int64  
 20  GOOD      1039 non-null   int6

In [5]:
# filtering dropping columns from dataset

column_drop = ['GameDate', 'AwayTeam','HomeTeam', 'qtr','min', 'sec', 'kickteam', 'def', 'down', 'togo', 'kicker', 'ydline', 'name','homekick', 'kickdiff', 'timerem','offscore','defscore','season', 'Blocked']
nfl_2 = nfl.drop(columns=column_drop)

print(nfl_2.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1039 entries, 0 to 1038
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   distance  1039 non-null   int64
 1   GOOD      1039 non-null   int64
 2   Missed    1039 non-null   int64
dtypes: int64(3)
memory usage: 24.5 KB
None


In [4]:
# Define the bins and labels
bins = [0, 10, 20, 30, 40, 50, 60, ]  # Adjust the thresholds as needed
labels = ['0-10', '10-20', '20-30', '30-40', '40-50']

# Create a new column with the bin labels
df['binned_column'] = pd.cut(df['numeric_column'], bins=bins, labels=labels)

# Group by the new binned column and calculate the mean of each group
grouped_df = df.groupby('binned_column').mean()

print(grouped_df)